<a href="https://colab.research.google.com/github/justalge/another_python_tutorial/blob/main/Lecture_12_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### В этом туториале мы попробуем получить ответ от [веб сервиса](http://www.cbs.dtu.dk/services/NetMHCpan/) внутри этого ноутбука, а не через браузер

Импортируем нужные модули:

In [ ]:
import requests

#### Ссылка на скрипт на сервере, который обрабатывает изначальный POST запрос:

Всегда, когда вы сабмитите форму в интернете, вы посылаете POST запрос скрипту адрес которого можно узнать просмотрев submitted form data. Загуглите "how to view submitted form data" или посмотрите вот это:

https://www.youtube.com/watch?v=SvUqk683mSA

https://wpscholar.com/blog/view-form-data-in-chrome/

In [ ]:
url = 'http://www.cbs.dtu.dk/cgi-bin/webface2.fcgi'

#### Формируем данные формы (словарь `post_dadta` - это я его так назвал) - как их формировать смотри так же, как адрес обрабатывающего скрипта (ссылки выше) - и отправляем с помощью `requests` этот POST запрос:

In [ ]:
post_data = {'configfile': '/usr/opt/www/pub/CBS/services/NetMHCpan-4.1/NetMHCpan.cf',
                'inp': 0,
                'SEQPASTE': 'NLVPMVATV',
                'master': 1,
                'thrs': 0.5,
                'thrw': 2,
                'threshold': -99
            }
r = requests.post(url, data=post_data)  # тут хранится ответ
type(r)  # видим, что этот ответ упакован в экземпляр класса Response,
         # который находится в папке (или файле) models, который находится в папке
         # (папка всего модуля) requests

requests.models.Response

In [1]:
# Вот кстати все методы (функции) и атрибуты (свойства),
# которые есть у этого объекта:

# dir(r)

#### Среди всех прочих есть метод `.content`, по которому получаем ответ сервера в виде строки, которая является `html` размеченным текстом

На этом движке взаимодействие с пользователем устроено так: сначала вы посылаете POST запрос и получаете ответ (это мы сделали). В этом ответе вам приходит информация о `jobid` - набор символов и букв, который вы (пользователи) посылаете уже другому скрипту-обработчику уже методом GET (просто в адресной строке). Мы достаем адрес второго обработчика из `r`, оттуда же достаем `jobid` и снова используем модуль `requests` для создания уже запроса GET:

In [ ]:
link = r.content[-120:-45]  # адрес второго обработчика (он такой же как и первый)
                            # с приделанными к нему данными методом GET - в форме
                            # 'key: value' после знака '?'
print(link)
jobid = link[-24:]
url2 = link[:-31]
print(jobid)
print(url2)

b'http://www.cbs.dtu.dk//cgi-bin/webface2.fcgi?jobid=6101365800005B22C53E54DA'
b'6101365800005B22C53E54DA'
b'http://www.cbs.dtu.dk//cgi-bin/webface2.fcgi'


#### Ну и наконец создаем новый запрос GET:

Почему именно такой запрос, я узнал опять же просмотрев submitted form data в браузере, как описано по ссылкам выше

In [ ]:
get_data = {'jobid': jobid,
            'wait' : '20'
            }
result = requests.get(url2, params=get_data)

In [ ]:
result.content  # тут находится нужный нам ответ в виде html строки

b'<html>\n<title> NetMHCpan 4.1 Server - prediction results</title>\n<body text="#000000" bgcolor="#f8f8f8"\n      link="#FF3399" vlink="#FF3399" alink="#808080">\n<font face="ARIAL,HELVETICA">\n<br>\n<table>\n<tr><td><img src="/images/m_logo.gif">\n    <td>&nbsp;&nbsp;&nbsp;\n    <td><h2>NetMHCpan Server - prediction results</h2>\n        <h3>Technical University of Denmark</h3>\n</table>\n<br>\n</font>\n<hr>\n<pre>\n\n# NetMHCpan version 4.1b\n\n# Tmpdir made /usr/opt/www/webface/tmp/server/netmhcpan/6101365800005B22C53E54DA/netMHCpanWfQpLk\n# Input is in FSA format\n\n# Peptide length 8,9,10,11\n\n# Make EL predictions\n\nHLA-A02:01 : Distance to training data  0.000 (using nearest neighbor HLA-A02:01)\n\n# Rank Threshold for Strong binding peptides   0.500\n# Rank Threshold for Weak binding peptides   2.000\n---------------------------------------------------------------------------------------------------------------------------\n Pos         MHC        Peptide      Core Of Gp Gl 

#### Сохраним его в виде `html` странице на сервере колаба:

In [ ]:
with open("results.html", "w") as f:
    f.write(str(result.text))
!ls  # удостоверимся, что сохранили

results.html  sample_data


#### Ну и запустим вебсервер в колабе, чтобы просмотреть наш результат как веб страницу - нужно нажать на появившуюся ссылку и выбрать потом наш `html` файл (чтобы делать потом что-то еще нужно будет остановить вебсервер, остановив ячейку):

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))
# https://z4spb7cvssd-496ff2e9c6d22116-8000-colab.googleusercontent.com/
!python -m http.server 8000

https://mp1ubn0bvld-496ff2e9c6d22116-8000-colab.googleusercontent.com/
Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [28/Jul/2021 11:10:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 11:10:46] code 404, message File not found
127.0.0.1 - - [28/Jul/2021 11:10:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Jul/2021 11:10:47] "GET /results.html HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 11:10:50] code 404, message File not found
127.0.0.1 - - [28/Jul/2021 11:10:50] "GET /images/m_logo.gif HTTP/1.1" 404 -
127.0.0.1 - - [28/Jul/2021 11:10:50] code 404, message File not found
127.0.0.1 - - [28/Jul/2021 11:10:50] "GET /favicon.ico HTTP/1.1" 404 -

Keyboard interrupt received, exiting.
^C


Все.